### Employee Exit Surveys (DETE & TAFE Institute in Queensland, Australia)

This project is contains dataset collect froman employee exit survey fron guided project, employees of the Department of Education, Training and Employment (DETE) and the Technical and Further Education (TAFE) institute in Queensland, Australia.The aim of this project is to clean and analyze this dataset. Both emmployee exit survey will be combined and analyzed together to answer the following questions:
- Are employees who only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction? 
- What about employees who have been there longer?
- Are younger employees resigning due to some kind of dissatisfaction? What about older employees?


In [ ]:
# Read import necessary libaries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dete_survey = pd.read_csv('dete_survey.csv') # Read the dete_survey csv file

tafe_survey = pd.read_csv('tafe_survey.csv') # Read the tafe_survey csv file

In [ ]:
dete_survey.info()

In [ ]:
dete_survey.head()

In [ ]:
print(dete_survey[['SeparationType']].value_counts())
print(dete_survey[['Age']].value_counts())
print(dete_survey[['Position']].value_counts())

In [ ]:
dete_survey.isnull().sum()

In [ ]:
tafe_survey.info()

In [ ]:
tafe_survey.head()

In [ ]:
tafe_survey['Institute'].value_counts()

In [ ]:
tafe_survey.isnull().sum()

### Findings From The Department of Education, Training and Employment (DETE) Survey

- The DETE employee exit survey reveals that most departures were due to age retirement and various forms of resignation, with the workforce largely composed of older employees, particularly those aged 56 and above. Teaching roles dominated the data, and common reasons for leaving included job dissatisfaction, interpersonal conflicts, and lack of recognition. Several data fields, especially around diversity and classification, were incomplete. To address these findings, DETE should focus on improving retention through better workplace support, succession planning for an aging workforce, and more consistent data collection—particularly in diversity and employee feedback areas.

### Findings From The Technical and Further Education (TAFE) Survey

- The dataset contains 702 employee exit records from various TAFE institute, with 72 columns covering demographics, reasons for leaving, contributing factors, workplace views, and induction experiences. Key fields include cessation year, reason for leaving, and satisfaction ratings. Some columns have significant missing data, especially in contributing factors and demographic sections. Most records come from a few major institutes. The next steps could involve cleaning the data, analyzing exit reasons, comparing institute satisfaction, or visualizing trends, depending on the analysis goals.Unlike the DETE survey, the data collection from the TAFE suvey was not standardized, hence, most of the column variables have different names and this most likely contributes to the majority of the missing data.

## Identify Missing Values and Drop Unnecesary Columns
The dete_survey dataframe includes 'Not Stated' entries that represent missing values, but they aren't recorded as NaN. Both dete_survey and tafe_survey contain many columns unnecessary for analysis. While the dataframes share similar information, the column names differ. Several columns also indicate that an employee resigned due to dissatisfaction. 
To begin, the plan is to convert 'Not Stated' to NaN using pd.read_csv() and then remove columns that aren't needed for analysis.

In [ ]:
# Drop missing columns from the dete survey dataset
dete_survey= pd.read_csv('dete_survey.csv', na_values= 'Not Stated')
# Drop missing columns in the dete_survey
dete_survey_updated= dete_survey.drop(dete_survey.columns[28:49], axis=1)
dete_survey_updated.info()

In [ ]:
# Drop missing columns in the tafe_survey
tafe_survey_updated= tafe_survey.drop(tafe_survey.columns[17:66], axis= 1)
tafe_survey_updated.info()

The missing values in the dete_survey was updated from `'Not Stated' to NaN` while also dropping the columns not needed for the analysis, about 21 columns was dropped leaving the total number of 34 columns from the initial 55 columns. Also, about 48 columns were dropped from the the tafe_survey leaving the total number of coulmns to 22.

### Clean Column Names
Each dataframe contains many of the same columns, but the column names are different. Below are some columns selected for use in the final analysis: 


| dete_survey | tafe_survey | Definition |
| :-----: | :-----: | :-----: |
| ID | Record ID | An id used to identify the participant of the survey |
| SeparationType | Reason for ceasing employment | The reason why the participant's employment ended |
| Cease Date | CESSATION YEAR | The year or month the participant's employment ended |
| DETE Start Date |  | The year the participant began employment with the DETE |
|  | LengthofServiceOverall. Overall Length of Service at Institute (in years) | The length of the person's employment (in years) (in years) | 
| Age | CurrentAge. Current Age | The age of the participant |
| Gender | Gender	Gender. What is your Gender? | The gender of the participant |

Since the goal is to combine the two datasets, it's necessary to standardize column names using the DataFrame.columns attribute.

In [ ]:
# Rename columns in dete_survey
dete_survey_updated.columns= dete_survey_updated.columns.str.lower().str.strip().str.replace(' ', '_')
dete_survey_updated.columns

In [ ]:
# Rename columns in tafe_survey
tafe_survey_updated= tafe_survey_updated.rename (columns= {'Record ID': 'id', 'CESSATION YEAR': 'cease_date', 'Reason for ceasing employment': 'separationtype', 'Gender. What is your Gender?': 'gender',
'CurrentAge. Current Age': 'age', 'Employment Type. Employment Type': 'employment_status', 'Classification. Classification': 'position',
'LengthofServiceOverall. Overall Length of Service at Institute (in years)': 'institute_service', 'LengthofServiceCurrent. Length of Service at current workplace (in years)': 'role_service'})
tafe_survey_updated.columns

### Column Name Standardization

To prepare the `dete_survey` and `tafe_survey` dataframes for analysis, column names were standardized:

- **In `dete_survey`:**  
  All column names were converted to lowercase, trailing whitespace was removed, and spaces were replaced with underscores. These steps improve naming consistency and simplify column referencing in code.

- **In `tafe_survey`:**  
  A custom mapping was used to rename selected columns to align with the naming format used in `dete_survey`. For example, `'Record ID'` became `'id'`, `'CESSATION YEAR'` became `'cease_date'`, among others. The updated names follow a consistent and readable style.

Standardizing column names across both datasets makes data handling easier and allows for accurate comparison.

### Filter The Data

The project's end goal is to answer the question of employees resigning due to dissatisfaction at the institute. However, the separationtype column in the `dete_survey_updated` dataframe includes several entries that mention resignation, such as:
- Resignation due to other reasons
- Resignation to join another employer
- Resignation for relocation overseas or interstate

These indicate different resignation circumstances among survey respondents.

So, it is important to take account of these variations without dropping useful data.


In [ ]:
# Count unique values in tafe_survey_updated

tafe_survey_updated['separationtype'].value_counts()

In [ ]:
# Count unique values in dete_survey_updated
dete_survey_updated['separationtype'].value_counts()

Since  the `dete_survey_updated` contains several entries that mention resignation, it is best to standardize to only carry `Resignation` so ensure all the values are counted without ommission.

In [ ]:
# Get total count of resignation
dete_survey_updated['separationtype']= dete_survey_updated['separationtype'].str.replace('-',' ').str.split().str[0]
dete_survey_updated['separationtype'].value_counts()

In [ ]:
# Filter data for both dataframes by resignation

dete_resignations= dete_survey_updated[dete_survey_updated['separationtype'] == 'Resignation'].copy()
tafe_resignations= tafe_survey_updated[tafe_survey_updated['separationtype'] == 'Resignation'].copy()

### Verify the Data
Before proceeding with further cleaning and analysis, it's important to check that the data is generally consistent. While not all errors can be detected, reviewing the data for obvious issues helps prevent completing an analysis that may be invalid due to poor data quality.

In this step, the focus is on ensuring that the values in the `cease_date` and `dete_start_date` columns appear reasonable. The `cease_date` represents the final year of employment, and the `dete_start_date` reflects the year employment began. It would be illogical for these dates to extend beyond the current year.

Additionally, considering that individuals in this profession typically begin their careers in their 20s, a `dete_start_date` earlier than 1940 would be highly unusual. A significant number of dates far outside a reasonable range could indicate serious problems with the dataset and may justify halting the analysis. However, if only a few outliers are present, it would be appropriate to remove them before continuing.

In [ ]:
# Check years for inconsistencie and clean columns in each dataframe
dete_resignations['cease_date'].value_counts()

In [ ]:
dete_resignations['cease_date']= dete_resignations['cease_date'].astype(str).str.split('/').str[-1]
dete_resignations['cease_date']= dete_resignations['cease_date'].astype(float)
dete_resignations['cease_date'].value_counts()

In [ ]:
dete_resignations['dete_start_date'].value_counts().sort_values()

In [ ]:
tafe_resignations['cease_date'].value_counts().sort_values()

In [ ]:
plt.boxplot([
    dete_resignations['cease_date'].dropna(),
    dete_resignations['dete_start_date'].dropna()
], labels=['Cease Date', 'Start Date'])

plt.title('Boxplot of Employment Dates')
plt.ylabel('Year')
plt.show()


In [ ]:
plt.boxplot(tafe_resignations['cease_date'].dropna())
plt.title('Boxplot of Cease Dates')
plt.ylabel('Year')
plt.show()

### Boxplot Analysis of Employment Dates
The boxplots highlight potential inconsistencies and outliers in the employment date columns: 
- In the first plot comparing `dete_start_date` and `cease_date`, the `dete_start_date` column shows a narrow range for most values, with a few clear outliers indicating unusually early start years. The `cease_date` column presents a broader distribution, along with several outliers below the lower whisker, suggesting possible data entry errors or atypical resignation years.

- The second plot, which likely represents calculated years of service, shows a more even distribution. However, an outlier on the lower end indicates an extremely short service period that may warrant further inspection. These visualizations support identifying and addressing anomalous values that could impact the reliability of further analysis.

- The DETE resignation data shows most resignations happened in 2012 and 2013, with 129 and 146 cases respectively, and minimal resignations in other years. In contrast, TAFE resignations are more evenly distributed, peaking in 2011 (116), followed by steady counts in 2012, 2010, and 2013. Since the aim of the project is not to analyze by year, this will be left at as it is.

### Create New Column
With the years in the `dete_resignations` dataframe verified, these values can now be used to create a new column. The objective is to explore whether employees with shorter tenures are resigning due to dissatisfaction, and how this compares to those with longer service periods. In human resources, this duration is commonly referred to as "years of service."

The `tafe_resignations` dataframe already includes a "service" column, previously renamed to `institute_service`. To enable a consistent analysis across both datasets, a corresponding `institute_service` column will be created in `dete_resignations`.

In [ ]:
#Create a new column for length of service for the dete_resignations dataframe
dete_resignations['institute_service']= dete_resignations['cease_date'] - dete_resignations['dete_start_date']
dete_resignations['institute_service'].head()

### Explanation For Creating The New Column
A new column called `institute_service` was added to the `dete_resignations` DataFrame by calculating the difference between the `cease_date` and `dete_start_date` columns. This change quantifies the length of service at the institute for each record. Calculating this difference helps analyze the duration employees worked before resigning, which can be useful for identifying trends or patterns in employee tenure. The use of `.head()` at the end provides a quick preview of the first few values in the new column to verify that the calculation was performed correctly.

### Identify Dissatisfied Employees
Below are the columns used to categorize employees as "dissatisfied" from each dataframe.

`tafe_survey_updated:`

- Contributing Factors. Dissatisfaction

- Contributing Factors. Job Dissatisfaction

`dete_survey_updated:`

- job_dissatisfaction

- dissatisfaction_with_the_department

- physical_work_environment

- lack_of_recognition

- lack_of_job_security

- work_location

- employment_conditions

- work_life_balance

- workload

If any of the factors listed above caused an employee to resign, they will be marked as dissatisfied in a new column.

To create the new column, the following steps will be taken:

- Convert the values in the `'Contributing Factors. Dissatisfaction' and 'Contributing Factors. Job Dissatisfaction'` columns in the `tafe_resignations` dataframe to True, False, or NaN values.

- If any of the specified columns contain a True value, the new column named `dissatisfied` will be set to True. To achieve this, a function will be written that:

    - Returns True if any element in the selected columns is True

    - Returns False if none of the elements in the selected columns is True

    - Returns NaN if the value is missing

After these changes, the `dissatisfied` column will only contain the following values:

- True: indicates resignation due to job dissatisfaction

- False: indicates resignation for reasons other than job dissatisfaction

- NaN: indicates missing data

In [ ]:
print(tafe_resignations['Contributing Factors. Dissatisfaction'].value_counts())
print(tafe_resignations['Contributing Factors. Job Dissatisfaction'].value_counts())

In [ ]:
def update_vals(x):
    if x == '-':
        return False
    elif pd.isnull(x):
        return np.nan
    else:
        return True
 
# Update the values in columns related to dissatisfaction to be either True, False, or NaN
tafe_resignations['dissatisfied'] = tafe_resignations[['Contributing Factors. Dissatisfaction', 'Contributing Factors. Job Dissatisfaction']].applymap(update_vals).any(axis=1)
tafe_resignations_up = tafe_resignations.copy()

In [ ]:
tafe_resignations_up['dissatisfied'].value_counts(dropna= False)

In [ ]:
# Update the values in columns related to dissatisfaction to be either True, False, or NaN

dete_resignations['dissatisfied']= dete_resignations[['job_dissatisfaction', 'dissatisfaction_with_the_department', 'physical_work_environment', 'lack_of_recognition','lack_of_job_security','work_location','employment_conditions','work_life_balance','workload']].any(axis=1, skipna= False)
dete_resignations_up = dete_resignations.copy() 

In [ ]:
dete_resignations_up['dissatisfied'].value_counts(dropna= False)

### Explanation of Changes
A new column named `'dissatisfied'` was added to both the `dete_resignations` and `tafe_resignations` DataFrames to indicate whether an employee experienced any form of job dissatisfaction. In the `dete_resignations` DataFrame, this column was created by evaluating multiple relevant columns—such as `job_dissatisfaction, workload, and lack_of_recognition` using `.any(axis=1, skipna=False)` to return True if any of the selected values in a row indicated `dissatisfaction`. 

In the `tafe_resignations` DataFrame, two dissatisfaction-related columns were processed with a custom function to convert values to True, False, or NaN. The results were then combined using `.any(axis=1)` to produce a single Boolean outcome. These changes make it easier to identify dissatisfied employees and perform consistent analysis across both datasets.

### Combine The Data
The datasets are now ready to be combined. The final objective is to aggregate the data based on the institute_service column, so the combined data should be structured in a way that supports straightforward aggregation by this column.

In [ ]:
# Create new column to differntiate both datasets
dete_resignations_up['institute']= 'DETE'
tafe_resignations_up['institute']= 'TAFE'

In [ ]:
# Combine both dataframe
combined= pd.concat([dete_resignations_up, tafe_resignations_up], axis=0)
combined.isnull().sum()

In [ ]:
# Drop columns with less than 500 null 
combined_updated= combined.dropna(thresh = 500, axis=1).copy()
combined_updated.isnull().sum()

A new column named `institute` was added to both dataframes to indicate the source of each resignation entry. The value `"DETE"` was assigned to all rows in the `dete_resignations_up` dataframe, and `"TAFE"` to all rows in the `tafe_resignations_up` dataframe. This allows for easy identification of the data's origin after combining the two datasets into a single dataframe called `combined`. 
To ensure the dataset contains only columns with sufficient data, the `dropna()` method was used with the thresh parameter to remove any columns with fewer than 500 non-null values. The resulting dataframe, `combined_updated`, retains only the most complete columns for further analysis.

### Clean The Service Column
Before analysis can begin, the institute_service column needs to be cleaned and standardized. This column contains values in different formats, making it difficult to interpret. To make the data usable, the values will be converted into defined career stage categories based on length of service. These categories are:

- New: Less than 3 years at a company

- Experienced: 3–6 years

- Established: 7–10 years

- Veteran: 11 or more years

This classification is informed by the idea that employee needs are better understood through career stage rather than age based on this [article](https://www.businesswire.com/newsroom).

In [ ]:
# Check unique values in the institute_service column
combined_updated['institute_service'].value_counts(dropna= False)

In [ ]:
# Change institute_service column to str
combined_updated['institute_service_up']= combined_updated['institute_service'].astype(str).str.extract(r'(\d+)')
combined_updated['institute_service_up']= combined_updated['institute_service_up'].astype(float)
combined_updated['institute_service_up'].value_counts()

In [ ]:
# Create function to segment the carrer categories

def career_cat(val):
    if pd.isnull(val):
        return np.nan
    if val < 3:
        return 'New'
    elif 3 <= val <= 6:
        return 'Experienced'
    elif 7 <= val <= 10:
        return 'Established'
    elif val >= 11:
        return 'Veteran'

# Apply the Series.apply() method

combined_updated['service_cat']= combined_updated['institute_service_up'].apply(career_cat)
combined_updated['service_cat'].head()